# Recurrent Neural Networks

## Introduction

Take a look a [this great article](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) for an introduction to recurrent neural networks and LSTMs in particular.

## Language Modeling

In this tutorial we will show how to train a recurrent neural network on a challenging task of language modeling. The goal of the problem is to fit a probabilistic model which assigns probabilities to sentences. It does so by predicting next words in a text given a history of previous words. For this purpose we will use the [Penn Tree Bank](http://www.cis.upenn.edu/~treebank/)(PTB) dataset, which is a popular benchmark for measuring quality of these models, whilst being small and relatively fast to train.

Language modeling is key to many interesting problems such as speech recognition, machine translation, or image captioning. It is also fun, too -- take a look [here](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

For the purpose of this tutorial, we will reproduce the results from [Zaremba et al., 2014](http://arxiv.org/abs/1409.2329)([pdf](http://arxiv.org/pdf/1409.2329.pdf)), which achieves very good results on the PTB dataset.

## Tutorial Files
This tutorial references the following files from __`models/rnn/ptb:`__

|    __File__    |                   __Purpose__                          |
| :-------------:|:------------------------------------------------------:|
| ptb_word_lm.py | The code to train a language model on the PTB dataset. |
| reader.py      | The code to reset the dataset.                         |

## Download and prepare the data
The data required for this tutorial is in the data/ directory of the PTB dataset from Tomas Mikolov's webpage: [http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz](https://www.google.com/url?q=http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz&usg=AFQjCNG0IP5OHusdIAdJIrrem-HMck9AzA)

The dataset is already preprocessed and contains overall 10000 different words, including the end-of-sentence marker and a special symbol (<unk>) for rare words. We convert all of them in the __reader.py__ to unique integer identifiers to make it easy for the neural network to process.

## The Model

### LSTM

The core of the model consists of an LSTM cell that processes one word at a time and computes probabilities of the possible continuations of the sentence. The memory state of the network is initialized with a vector of zeros and gets updated after reading each word. Also, for computational reasons, we will process data in mini-batches of size __`batch_size`__.

The basic pseudocode looks as follows:

### Truncated Backpropogation

In order to make the learning process tractable, it is a common practice to truncate the gradients for backpropagation to a fixed number (__`num_steps`__) of unrolled steps. This is easy to implement by feeding inputs of length `num_steps` at a time and doing backward pass after each iteration.

A simplified version of the code for the graph creation for truncated backpropagation:

And this is how to implement an iteration over the whole dataset:

### Inputs

The word IDs will be embedded into a dense representation (see the [Vector Representations Tutorial](https://www.tensorflow.org/versions/r0.9/tutorials/word2vec/index.html)) before feeding to the LSTM. This allows the model to efficiently represent the knowledge about particular words. It is also easy to write:

The embedding matrix will be initialized randomly and the model will learn to differentiate the meaning of words just by looking at the data.

### Loss Function

We want to minimize the average negative log probability of the target words:

$$ loss = -\frac{1}{N}\sum_{i=1}^{N} ln p_{target_i} $$

It is not very difficult to implement but the function __`sequence_loss_by_example`__ is already available, so we can just use it here.

The typical measure reported in the papers is average per-word perplexity (often just called perplexity), which is equal to

$$ e^{-\frac{1}{N}\sum_{i=1}^{N} ln p_{target_i}} = e^{loss} $$

and we will monitor its value throughout the training process.

## Stacking multiple LSTMs

To give the model more expressive power, we can add multiple layers of LSTMs to process the data. The output of the first layer will become the input of the second and so on.

We have a class called __`MultiRNNCell`__ that makes the implementation seamless:

## Run the Code

We are assuming you have already installed via the pip package, have cloned the tensorflow git repository, and are in the root of the git tree. (If [building from source](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/g3doc/get_started/os_setup.md#installing-from-sources), build the tensorflow/models/rnn/ptb:ptb_word_lm target using [bazel](https://github.com/bazelbuild/bazel)).

Next: __`cd tensorflow/models/rnn/ptb python ptb_word_lm --data_path=/tmp/simple-examples/data/ --model small`__

There are 3 supported model configurations in the tutorial code: "small", "medium" and "large". The difference between them is in size of the LSTMs and the set of hyperparameters used for training.

The larger the model, the better results it should get. The small model should be able to reach perplexity below 120 on the test set and the large one below 80, though it might take several hours to train.

## What Next?

There are several tricks we haven't mentioned that make the model better, including:
- decreasing learning rate schedule,

- dropout between the LSTM layers.

Study the code and modify it to improve th model even further.